In [1]:
import os 

In [2]:
%pwd  

'c:\\Users\\A.DUSHYANTH\\OneDrive\\Documents\\Text-Summarizer\\research'

In [3]:
os.chdir("../")  

In [4]:
%pwd

'c:\\Users\\A.DUSHYANTH\\OneDrive\\Documents\\Text-Summarizer'

In [5]:
from dataclasses import dataclass
from pathlib import Path


@dataclass(frozen=True)
class ModelTrainerConfig:
    root_dir: Path
    data_path: Path
    model_ckpt: Path
    num_train_epochs: int
    warmup_steps: int
    per_device_train_batch_size: int
    weight_decay: float
    logging_steps: int
    evaluation_strategy: str
    eval_steps: int
    save_steps: float
    gradient_accumulation_steps: int

In [6]:
from textSummarizer.constants import *
from textSummarizer.utils.common import read_yaml, create_directories

In [7]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)

        create_directories([self.config.artifacts_root])


    
    def get_model_trainer_config(self) -> ModelTrainerConfig:
        config = self.config.model_trainer
        params = self.params.TrainingArguments

        create_directories([config.root_dir])

        model_trainer_config = ModelTrainerConfig(
            root_dir=config.root_dir,
            data_path=config.data_path,
            model_ckpt = config.model_ckpt,
            num_train_epochs = params.num_train_epochs,
            warmup_steps = params.warmup_steps,
            per_device_train_batch_size = params.per_device_train_batch_size,
            weight_decay = params.weight_decay,
            logging_steps = params.logging_steps,
            evaluation_strategy = params.evaluation_strategy,
            eval_steps = params.evaluation_strategy,
            save_steps = params.save_steps,
            gradient_accumulation_steps = params.gradient_accumulation_steps
        )

        return model_trainer_config

In [8]:
from transformers import TrainingArguments, Trainer
from transformers import DataCollatorForSeq2Seq
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer
from datasets import load_dataset, load_from_disk
import torch

[2024-04-21 20:23:19,679: INFO: config: PyTorch version 2.2.2 available.]
[2024-04-21 20:23:19,684: INFO: config: TensorFlow version 2.14.0 available.]


In [9]:
class ModelTrainer:
    def __init__(self, config: ModelTrainerConfig):
        self.config = config


    
    def train(self):
        device = "cuda" if torch.cuda.is_available() else "cpu"
        tokenizer = AutoTokenizer.from_pretrained(self.config.model_ckpt)
        model_pegasus = AutoModelForSeq2SeqLM.from_pretrained(self.config.model_ckpt).to(device)
        seq2seq_data_collator = DataCollatorForSeq2Seq(tokenizer, model=model_pegasus)
        
        #loading data 
        dataset_samsum_pt = load_from_disk(self.config.data_path)

        # trainer_args = TrainingArguments(
        #     output_dir=self.config.root_dir, num_train_epochs=self.config.num_train_epochs, warmup_steps=self.config.warmup_steps,
        #     per_device_train_batch_size=self.config.per_device_train_batch_size, per_device_eval_batch_size=self.config.per_device_train_batch_size,
        #     weight_decay=self.config.weight_decay, logging_steps=self.config.logging_steps,
        #     evaluation_strategy=self.config.evaluation_strategy, eval_steps=self.config.eval_steps, save_steps=1e6,
        #     gradient_accumulation_steps=self.config.gradient_accumulation_steps
        # ) 


        trainer_args = TrainingArguments(
             output_dir='pegasus-samsum', num_train_epochs=1, warmup_steps=500,
            per_device_train_batch_size=1, per_device_eval_batch_size=1,
             weight_decay=0.01, logging_steps=10,
             evaluation_strategy='steps', eval_steps=500, save_steps=1e6,
             gradient_accumulation_steps=16
        )

        trainer = Trainer(model=model_pegasus, args=trainer_args,
                  tokenizer=tokenizer, data_collator=seq2seq_data_collator,
                  train_dataset=dataset_samsum_pt["test"],
                  eval_dataset=dataset_samsum_pt["validation"])
        
        trainer.train()

        ## Save model
        model_pegasus.save_pretrained(os.path.join(self.config.root_dir,"pegasus-samsum-model"))
        ## Save tokenizer
        tokenizer.save_pretrained(os.path.join(self.config.root_dir,"tokenizer"))   

In [10]:
try:
    config = ConfigurationManager()
    model_trainer_config = config.get_model_trainer_config()
    model_trainer_config = ModelTrainer(config=model_trainer_config)
    model_trainer_config.train()
except Exception as e:
    raise e

[2024-04-21 20:23:26,521: INFO: common: yaml file: config\config.yaml loaded successfully]
[2024-04-21 20:23:26,521: INFO: common: yaml file: params.yaml loaded successfully]
[2024-04-21 20:23:26,521: INFO: common: created directory at: artifacts]
[2024-04-21 20:23:26,529: INFO: common: created directory at: artifacts/model_trainer]


Some weights of PegasusForConditionalGeneration were not initialized from the model checkpoint at google/pegasus-cnn_dailymail and are newly initialized: ['model.decoder.embed_positions.weight', 'model.encoder.embed_positions.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


  0%|          | 0/51 [00:00<?, ?it/s]

{'loss': 3.1101, 'grad_norm': 31.9448184967041, 'learning_rate': 1.0000000000000002e-06, 'epoch': 0.2}
{'loss': 3.0466, 'grad_norm': 15.821706771850586, 'learning_rate': 2.0000000000000003e-06, 'epoch': 0.39}
{'loss': 3.1612, 'grad_norm': 9.896061897277832, 'learning_rate': 3e-06, 'epoch': 0.59}
{'loss': 2.9898, 'grad_norm': 19.90555191040039, 'learning_rate': 4.000000000000001e-06, 'epoch': 0.78}
{'loss': 2.8523, 'grad_norm': 33.813507080078125, 'learning_rate': 5e-06, 'epoch': 0.98}


Some non-default generation parameters are set in the model config. These should go into a GenerationConfig file (https://huggingface.co/docs/transformers/generation_strategies#save-a-custom-decoding-strategy-with-your-model) instead. This warning will be raised to an exception in v4.41.
Non-default generation parameters: {'max_length': 128, 'min_length': 32, 'num_beams': 8, 'length_penalty': 0.8, 'forced_eos_token_id': 1}


{'train_runtime': 5248.7233, 'train_samples_per_second': 0.156, 'train_steps_per_second': 0.01, 'train_loss': 3.0227226416269937, 'epoch': 1.0}


In [12]:
!pip install --upgrade accelerate
!pip uninstall -y transformers accelerate
!pip install transformers accelerate

   ---------------------------------------- 0.0/297.6 kB ? eta -:--:--
   - -------------------------------------- 10.2/297.6 kB ? eta -:--:--
   -------- ------------------------------ 61.4/297.6 kB 825.8 kB/s eta 0:00:01
   ------------ -------------------------- 92.2/297.6 kB 871.5 kB/s eta 0:00:01
   ----------------------- ---------------- 174.1/297.6 kB 1.1 MB/s eta 0:00:01
   --------------------------- ------------ 204.8/297.6 kB 1.0 MB/s eta 0:00:01
   ---------------------------------- ----- 256.0/297.6 kB 1.1 MB/s eta 0:00:01
   -------------------------------------- 297.6/297.6 kB 967.4 kB/s eta 0:00:00



[notice] A new release of pip is available: 23.3 -> 24.0
[notice] To update, run: python.exe -m pip install --upgrade pip


Found existing installation: transformers 4.40.0
Uninstalling transformers-4.40.0:
  Successfully uninstalled transformers-4.40.0
Found existing installation: accelerate 0.29.3
Uninstalling accelerate-0.29.3:
  Successfully uninstalled accelerate-0.29.3
  Using cached transformers-4.40.0-py3-none-any.whl.metadata (137 kB)
  Using cached accelerate-0.29.3-py3-none-any.whl.metadata (18 kB)
Using cached transformers-4.40.0-py3-none-any.whl (9.0 MB)
Using cached accelerate-0.29.3-py3-none-any.whl (297 kB)



[notice] A new release of pip is available: 23.3 -> 24.0
[notice] To update, run: python.exe -m pip install --upgrade pip
